# Libraries

In [60]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
import torch
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Import data

In [69]:
# For training the model
X_train = pd.read_csv("../../X/train_v1.csv", index_col=[0])

# For cross validation
X_valid = pd.read_csv("../../X/valid_v1.csv", index_col=[0])

# For prediction
X_test = pd.read_csv("../../X/test_v1.csv", index_col=[0])

y_train_t2 = pd.read_csv("../../Task2/Y_train.csv")
y_valid_t2 = pd.read_csv("../../Task2/Y_valid.csv")
y_train_t2_value=y_train_t2["los_icu"]
y_valid_t2_value=y_valid_t2["los_icu"]

final_index = X_test.index

In [70]:
# Impute the 0 with mean
imp = SimpleImputer(missing_values=0, strategy='mean')
X_train_nozero = pd.DataFrame(imp.fit_transform(X_train))
X_valid_nozero = pd.DataFrame(imp.fit_transform(X_valid))
X_test_nozero = pd.DataFrame(imp.fit_transform(X_test))
X_train_nozero

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
0,-2.039985,-1.109747,1.170093,1.478921,-1.069743,0.090101,0.792807,-0.638602,-0.166695,-0.002273,...,-0.586358,-1.694872,-0.397490,-0.675987,-1.979855,0.000309,-0.124820,0.442871,0.869070,0.381991
1,0.793194,0.702851,-0.061390,0.198395,-1.044218,1.369469,1.737977,1.537514,-0.390618,-0.518339,...,-0.392555,-1.444821,-0.658260,-1.097709,-0.251806,0.000309,-0.098064,-0.812489,2.806903,1.307294
2,-1.561656,0.145354,0.546114,0.161234,-0.858553,-0.739112,-0.439621,1.723172,-0.472836,0.101098,...,1.865413,1.019964,1.590880,2.563201,0.061017,0.000309,-0.048254,0.100500,-0.476014,-1.941549
3,0.728804,-0.151030,-0.029561,-0.211957,-0.079990,-0.128434,-0.439621,-0.661305,-0.103117,-0.188482,...,1.029896,0.484141,0.833018,0.031234,0.061017,0.000309,-0.098064,-0.015089,0.071139,0.006806
4,-1.074128,-0.181217,-0.101854,-0.341346,-0.726826,0.186551,0.348021,0.603034,-0.194262,-0.157922,...,0.560349,-1.694872,0.083304,0.031234,0.061017,-1.333059,0.234019,-0.015089,0.823474,0.006806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16755,0.039265,-0.036621,-1.432434,-0.894033,-0.598336,-0.575247,-0.200239,0.667703,-0.559890,-1.013160,...,0.691546,0.091204,0.727080,0.355887,1.044231,0.000309,-0.098064,-0.127747,-0.236635,-0.070380
16756,0.039265,0.325411,0.370595,-0.841322,-0.004140,-0.420184,-0.331513,-0.389591,-0.197164,-0.067515,...,-0.513394,-2.230694,-0.715303,-0.003025,0.061017,0.000309,-0.098064,-0.263269,0.481503,0.265471
16757,0.039265,0.006264,-0.518391,-0.629821,-0.389777,-0.131727,0.116362,-1.128922,0.071254,-0.353408,...,-0.979489,1.448622,-0.583560,0.031234,1.044231,0.000309,-0.098064,-0.015089,-0.270832,0.006806
16758,0.039265,0.052558,1.023920,-0.077771,1.028054,-0.491283,-0.219544,2.021813,-0.559890,0.149653,...,-0.468511,-0.051682,-0.560471,0.355887,0.061017,0.000309,-0.098064,5.007816,-0.937674,-2.976518


In [75]:
scaler = preprocessing.StandardScaler()

X_train=X_train.values
X_train= scaler.fit_transform(X_train)
X_train=pd.DataFrame(X_train)

X_valid=X_valid.values
X_valid= scaler.fit_transform(X_valid)
X_valid=pd.DataFrame(X_valid)

X_test=X_test.values
X_test= scaler.fit_transform(X_test)
X_test=pd.DataFrame(X_test)

In [76]:

X_train_norm = X_train
X_valid_norm = X_valid
X_test_norm = X_test

print(X_train_norm.shape)
print(X_valid_norm.shape)
print(X_test_norm.shape)


(16760, 51)
(2394, 51)
(4790, 55)


In [68]:
y_train_t2_value

0        6.064560
1        3.672917
2        5.323056
3        1.584410
4        7.634815
           ...   
16755    1.703391
16756    2.731933
16757    7.949259
16758    3.142616
16759    2.161481
Name: los_icu, Length: 16760, dtype: float64

# Evaluation Matrix

In [65]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

# Hyperparameter and setting tuning

## Layer Choices

In [66]:
layer_list = []
rmse_lst=[]

for x in range (1,10,1):
    for y in range (1,20,1):
        layer_list.append((x,y))

for j in tqdm(range(len(layer_list))):
    i = layer_list[j]
    clf = MLPRegressor(solver="adam",
                       alpha=1e-5,
                       hidden_layer_sizes=i,
                       activation='relu',
                       random_state=3612,
                       max_iter=1000)

    clf.fit(X_train_norm, y_train_t2_value)
    y_pred = clf.predict(X_valid_norm)
    rmse1 = rmse(y_pred, y_valid_t2_value)
    rmse_lst.append(rmse1)
    
print(layer_list)
index = rmse_lst.index(min(rmse_lst))
plt.plot(list(range(len(layer_list))), rmse_lst, label='Layers Tuning')
plt.xlabel('Layer Choises')
plt.ylabel('RMSE')
plt.title(f"Layer {layer_list[index]} - ROC curve")
plt.legend(loc="best")
plt.annotate(np.min(rmse_lst).round(4), (np.argmin(rmse_lst), np.min(rmse_lst)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

  0%|                                                   | 0/171 [00:00<?, ?it/s]


ValueError: Found input variables with inconsistent numbers of samples: [2394, 16760]

In [ ]:
best_layernumber = layer_list[index]
best_layernumber

## Solver Choices

In [ ]:
solver_list = ["lbfgs","sgd","adam"]
rmse_lst=[]

for i in solver_list:
    clf = MLPRegressor(solver=i,
                        alpha=1e-5,
                        hidden_layer_sizes=best_layernumber,
                        random_state=1,
                        activation='relu',
                        max_iter=1000)

    clf.fit(X_train_norm, y_train_t2_value)
    y_pred = clf.predict(X_valid_norm)
    rmse1 = rmse(y_pred, y_valid_t2_value)
    rmse_lst.append(rmse1)
    
print(solver_list)
index = rmse_lst.index(min(rmse_lst))
plt.plot(solver_list, rmse_lst, label='Solver Tuning')
plt.xlabel('Solver Choises')
plt.ylabel('RMSE')
plt.title(f"Solver {solver_list[index]} - ROC curve")
plt.legend(loc="best")
plt.annotate(np.min(rmse_lst).round(4), (np.argmin(rmse_lst), np.min(rmse_lst)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
index = np.argmin(rmse_lst)
best_solver = solver_list[index]
best_solver

## Learning rate tuning

In [ ]:
lr_list = [1e-10,0.0001]+list(np.linspace(0.001,0.01,5))+list(np.linspace(0.01,0.1,5))+list(np.linspace(0.1,1,5))
rmse_lst=[]

for i in lr_list:
    clf = MLPRegressor(solver=best_solver,
                        hidden_layer_sizes=best_layernumber,
                        learning_rate_init=i,
                        learning_rate="adaptive", 
                        alpha=1e-5,
                        random_state=1,
                        activation='relu',
                        max_iter=1000)

    clf.fit(X_train_norm, y_train_t2_value)
    y_pred = clf.predict(X_valid_norm)
    rmse1 = rmse(y_pred, y_valid_t2_value)
    rmse_lst.append(rmse1)
    
print(lr_list)
index = rmse_lst.index(min(rmse_lst))
plt.plot(lr_list, rmse_lst, label='Learning Rate Tuning')
plt.xlabel('Learning Rate Choises')
plt.ylabel('RMSE')
plt.title(f"Learning Rate {lr_list[index]} - ROC curve")
plt.legend(loc="best")
plt.annotate(np.min(rmse_lst).round(4), (np.argmin(rmse_lst), np.min(rmse_lst)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()
print(np.min(rmse_lst).round(4))

In [ ]:
index = np.argmin(rmse_lst)
best_lr_choice =  lr_list[index]
best_lr_choice

## Regularization strength tuning

In [ ]:
reg_list = list(np.linspace(1e-5,0.00005,20))
rmse_lst=[]

for i in reg_list:
    clf = MLPRegressor(solver=best_solver,
                        hidden_layer_sizes=best_layernumber,
                        learning_rate_init=best_lr_choice,
                        alpha=i,
                        random_state=1,
                        activation='relu',
                        max_iter=1000)

    clf.fit(X_train_norm, y_train_t2_value)
    y_pred = clf.predict(X_valid_norm)
    rmse1 = rmse(y_pred, y_valid_t2_value)
    rmse_lst.append(rmse1)
    
print(reg_list)
index = rmse_lst.index(min(rmse_lst))
plt.plot(list(range(len(reg_list))), rmse_lst, label='Regularization strength tuning')
plt.xlabel('Regularization strength Choices')
plt.ylabel('RMSE')
plt.title(f"Regularization strength tuning {reg_list[index]} - ROC curve")
plt.legend(loc="best")
plt.annotate(np.min(rmse_lst).round(4), (np.argmin(rmse_lst), np.min(rmse_lst)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

In [ ]:
index = np.argmin(rmse_lst)
best_reg_choice =  reg_list[index]
best_reg_choice

In [ ]:
index = np.argmin(rmse_lst)
best_rs_choice =  rs_list[index]
best_rs_choice

## Activation function Choices

In [ ]:
activation_list = ['identity','logistic','tanh','relu']
rmse_lst=[]

for i in activation_list:
    clf = MLPRegressor(solver=best_solver,
                        hidden_layer_sizes=best_layernumber,
                        learning_rate_init=best_lr_choice,
                        alpha=best_reg_choice,
                        random_state=3612,
                        activation=i,
                        max_iter=1000)

    clf.fit(X_train_norm, y_train_t2_value)
    y_pred = clf.predict(X_valid_norm)
    rmse1 = rmse(y_pred, y_valid_t2_value)
    rmse_lst.append(rmse1)
    
print(activation_list)
index = rmse_lst.index(min(rmse_lst))
plt.plot(activation_list, rmse_lst, label=' Random State Tuning')
plt.xlabel(' Random State Choices')
plt.ylabel('RMSE')
plt.title(f" Activation Tuning {activation_list[index]} - ROC curve")
plt.legend(loc="best")
plt.annotate(np.min(rmse_lst).round(4), (np.argmin(rmse_lst), np.min(rmse_lst)), xycoords="data",
                 xytext=(40, 20), textcoords="offset pixels", arrowprops=dict(facecolor="black", shrink=0.1), fontsize=10,
                 horizontalalignment="center", verticalalignment="top")
plt.show()

# Test

In [ ]:
X = np.concatenate((X_train_norm,X_valid_norm),axis=0)
y = np.concatenate((y_train_t2_value,y_valid_t2_value),axis=0)

In [ ]:
# Final model
reg1 = MLPRegressor(
                   )

reg1.fit(X_train_norm,y_train_t2_value)
y_pred = reg1.predict(X_valid_norm)
rmse(y_pred, y_valid_t2_value)

In [ ]:
y_pred = reg1.predict(X_valid_norm)
rmse(y_pred, y_valid_t2_value)

In [ ]:
X_test_norm.describe()

In [ ]:
X_test

In [ ]:

result = reg1.predict(X_test_norm)
res1 = pd.DataFrame(result.tolist(), columns=["Predicted"])
res1.index = final_index
res1.index.name = 'Id'
pd.DataFrame(res1).to_csv('out12col.csv')
#print(torch.count_nonzero(torch.from_numpy(np.array(res1)).to(torch.float32)))
res1